# PASO A PASO
1. Importar librerías

In [1]:
import psycopg2
import psycopg2.extras
#connection = None
#cursor = None

2. Generar conexión

In [2]:
connection = psycopg2.connect(
        host= 'localhost',
        dbname = "proyectoX",
        user= "postgres" ,
        password = 'PRUEBA123',
        port = 5433)

3. Generar consultas sql
- ¿Total de ventas que se hicieron en mayo y junio para los años 2021 y 2022 en la tabla customers?
## Solución con SQL

In [5]:
statement = """Select * FROM(SELECT CAST(SUM(price::numeric::int * quantity) as money) AS total_sales,
	DATE_PART('MONTH', fecha) as month_date,
	DATE_PART('YEAR', fecha)as year_date 
	FROM customers
	GROUP BY month_date,year_date) as date_table
    WHERE month_date IN ('5','6') and
    year_date  IN ('2022','2021')"""

In [6]:
cursor = connection.cursor(cursor_factory=psycopg2.extras.DictCursor) #retorna el cursor no como tupla sino como diccionario
cursor.execute(statement)
rows = cursor.fetchall()
for row in rows:
    print(row)

['$ 52.602,00', 5.0, 2022.0]
['$ 148.334,00', 6.0, 2022.0]


2. ¿Cuales son los datos duplicados en la tabla customers?
## Solución con SQL

In [3]:
statement_2 = """ 
SELECT *
FROM (
	SELECT  *,id,
	ROW_NUMBER() OVER(
	PARTITION BY
		first_name,
		last_name,
		email,
		gender,
		product,
		quantity,
		fecha
	ORDER BY id ASC) as row
	FROM customers)AS duplicados
WHERE duplicados.row > 1;
"""""

In [4]:
cursor = connection.cursor(cursor_factory=psycopg2.extras.DictCursor) #retorna el cursor no como tupla sino como diccionario
cursor.execute(statement_2)
rows = cursor.fetchall()
for row in rows:
    print(row)

[103, 'Whit', 'Hayball', 'whayball0@spotify.com', 'Male', 'Wine-Red,Antinori Santa', '$ 899,00', 1, datetime.datetime(2022, 8, 9, 0, 0), 103, 2]


3.  ¿Muestreme las incoherencias de las columnas product entre las tablas customers y product?

In [21]:
statement_3 = """
SELECT 
    customers.product product_table_1,
	product.product as product_table_2
FROM customers
LEFT JOIN product ON
	customers.product = product.product
WHERE product.product is not null"""

In [22]:
cursor = connection.cursor(cursor_factory=psycopg2.extras.DictCursor) #retorna el cursor no como tupla sino como diccionario
cursor.execute(statement_3)
rows = cursor.fetchall()
for row in rows:
    print(row)

['Ecolab Silver Fusion', 'Ecolab Silver Fusion']
['Flour - Masa De Harina Mexican', 'Flour - Masa De Harina Mexican']
['Stock - Veal, Brown', 'Stock - Veal, Brown']
['Salsify, Organic', 'Salsify, Organic']
